In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
file_location = 'Resources/18_19'

### Master Course Info
#### Double periods, Multiple Course Instances, Min, Max, HS/MS Category, Required Grades, Room Type

This file is constructed by the school

In [3]:
course_info = pd.read_excel(file_location + '/Input1_Refactored.xlsx', delimiter=',')

#### Teacher Info

Using the first input file from the school, we constuct a file template for the school to input teacher/course matches 
This output file is "Teacher Template"

In [4]:
courses = pd.DataFrame(course_info['Course Name'], columns = ['Course Name', 'Teacher Name'])

# add multiple instances of courses to teacher doc
add = pd.DataFrame() 

# iterate over course with multiple instances
for index, row in course_info.loc[course_info['Number of Instances'] >= 1].iterrows():
    instances = np.arange(row['Number of Instances']) + 1
    for i in instances[1:]: # add a course for every additional instance
        c = row['Course Name'] + ' V'+ str(int(i))
        updated_row = row.copy()
        updated_row['Course Name'] = c
        add = add.append(updated_row, ignore_index=True)

courses = pd.concat([pd.DataFrame(add['Course Name'], columns = ['Course Name']), courses], axis=0).sort_values(by=['Course Name'])
courses.reset_index(drop=True).to_csv(file_location + '/Teacher_Template.csv', index=False)

### Construct Input Files for LP Solver
#### Course Room Requirements, (Max, Min, MS Category, HS Category, Double, Teacher Info) 

Using the info from the school, construct input files that are readable by our LP program

In [5]:
all_info = pd.concat([course_info, add], axis=0).sort_values(by=['Course Name']).reset_index(drop=True)

# add in double periods 
add_double = pd.DataFrame() 
double = all_info.copy()
double.dropna(subset=['Double Period'])
for index, row in all_info.loc[all_info['Double Period'] == 'Yes'].iterrows():
    c = row['Course Name'] + ' II'
    updated_row = row.copy()
    updated_row['Course Name'] = c
    updated_row['Double Period'] = 0
    add_double = add_double.append(updated_row, ignore_index=True)    
add_double
all_info = pd.concat([all_info, add_double], axis=0).sort_values(by=['Course Name'])

# convert "yes" to 1, keep 0
all_info['Double Period'] = all_info['Double Period'].map({'Yes': 1, 0:0})
all_info.reset_index(drop=True)
all_info.to_csv(file_location + '/LP_Input.csv', index=False)

#### Proximity Matrix

From the input file above, construct the proximity matrix

In [6]:
def dept_proximity(file_location, file_name):
    # columns are department, rows are course names
    info = pd.read_csv(file_location + file_name, delimiter=',')

    depts = info[["Course Name", "MS Category", "HS Category"]]
    depts = depts.fillna(0)

    # iterate over depts
    msd = set(depts["MS Category"])
    msd.remove(0)

    hsd = set(depts["HS Category"])
    hsd.remove(0)

    cols = msd.union(hsd)
    single = [subject for subject in cols if '&' not in subject]
    double = [subject for subject in cols if '&' in subject]

    sim = pd.DataFrame(index=depts['Course Name'], columns=single)

    # 2nd period of double courses
    second = np.array(info.loc[info['Double Period'] == 0]['Course Name'])

    for d in cols:  
        dept_courses_MS = depts["Course Name"].loc[depts["MS Category"] == d]
        dept_courses_HS = depts["Course Name"].loc[depts["HS Category"] == d]
        dept_courses = dept_courses_MS.append(dept_courses_HS, ignore_index=True)
        for course in dept_courses:

            # set all 2nd part in double period to 0
            if course in second:
                if '&' in d:
                    d1 = d.split(" ")[0]
                    d2 = d.split(" ")[2]

                    sim.loc[course, d1]=0
                    sim.loc[course, d2]=0

                else:
                    sim.loc[course, d] = 0

            elif d in single: 
                sim.loc[course, d]=1

            else: #d in double
                # get two depts
                d1 = d.split(" ")[0]
                d2 = d.split(" ")[2]

                sim.loc[course, d1]=1
                sim.loc[course, d2]=1


    sim = sim.fillna(0)            
    sim.to_csv(file_location + '/Proximity.csv')

    
dept_proximity(file_location, '/LP_Input.csv')